In [11]:
import chess
import chess.pgn
import requests
import io
import json
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

In [12]:
%run ../bedrock_setup.py

In [13]:
from langchain.chat_models import init_chat_model

model = init_chat_model("us.amazon.nova-micro-v1:0", # "us.anthropic.claude-3-5-haiku-20241022-v1:0"
                        model_provider="bedrock_converse",
                        region_name="us-east-1",
                        client=bedrock_client)

In [47]:
@tool
def fetch_puzzle(query: str) -> dict:
    """
    Get a random puzzle from chess.com in a json format.
    """
    url = "https://api.chess.com/pub/puzzle/random"
    headers = {'User-Agent': 'My Python App'}
    response = requests.get(url, headers=headers).json()

    return response

@tool
def get_info(response: dict) ->  dict:
    """
    Get a FEN from the response on chess.com.
    """
    pgn_io = io.StringIO(response['pgn'])
    game = chess.pgn.read_game(pgn_io)

    info = {
        'title' : response['title'],
        'fen' : response['fen'],
        'answer' : str(game.mainline_moves())
    }
    
    return info

In [48]:
tools = [fetch_puzzle, get_info]

In [49]:
prompt = """
You are an intelligent agent that helps users to get a chess puzzle.
Follow these steps to answer user queries:
1. Use the `fetch_puzzle` tool to request a puzzle using chess.com API.
2. Use the `get_info` tool to retrieve relevant information of the puzzle.
4. If you can't find enough information, state that explicitly.

Never include explanations, steps, or any text besides the dictionary info or error message.
Only provide a dictoinary from the get_info tool.
"""

In [50]:
agent_executor = create_react_agent(model, tools, prompt = prompt)

In [51]:
query = "Get me a random chess puzzle from chess.com"
response = agent_executor.invoke({"messages": [HumanMessage(content=query)]})

In [52]:
response["messages"][-1].content

'{\n  "title": "Forced into a Box",\n  "fen": "8/8/6N1/8/6p1/5Bpk/4K2b/7r w - - 0 1",\n  "answer": "1. Bg2+ Kxg2 2. Nf4+ Kg1 3. Ke1 g2 4. Ne2#"\n}'